<a href="https://colab.research.google.com/github/jiya07/Multiclass-Sentiment-Classification-Using-BERT/blob/main/Multiclass_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The multiclass sentiment analysis of consumer reviews from mouthshut.com is done using Bidirectional Encoder Representations from Transformers or **BERT** which is a very popular approach in todays world. 

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.2
tensorflow_hub version :  0.12.0


In [3]:
!pip install bert-tensorflow==1.0.1
#!pip install bert-tensorflow

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
from google.colab import files

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fileDownloaded = drive.CreateFile({'id':'19RhXfgmmMKruWpPEgAImsXb-efImC1C9'})
fileDownloaded.GetContentFile('df_reviews.csv')
fileDownloaded1 = drive.CreateFile({'id':'1kF8T2g8rHvPrDUZK53pK2u8U4tPVCK_u'})
fileDownloaded1.GetContentFile('df_site_reviews.csv')

In [7]:
df_reviews = pd.read_csv('df_reviews.csv', encoding = "ISO-8859-1")
df_site_reviews = pd.read_csv('df_site_reviews.csv', encoding = "ISO-8859-1")
df_reviews=df_reviews.rename(columns={"reviewId": "ReviewId","review_text":"Review","rating":"Rating"})
df_site_reviews.drop(['Rating'], axis = 1, inplace = True)

df = pd.merge(df_site_reviews,df_reviews, on=['ReviewId', 'ReviewId'], how='inner')
df.drop(['Recommends','Reviews','title_url','date','views'], axis = 1, inplace = True)
#df_merged=pd.merge(df_reviews,df_site_reviews,on='ReviewId')

#df_merged = pd.merge(df_reviews, df_reviews,  how='left', left_on=['reviewId'], right_on = ['ReviewId'])

#df = pd.merge(df_reviews, df_site_reviews, on=['ReviewId', 'ReviewId'], how='inner')
#df_delivery=df_merged['title_text'].str.lower().contains("delivery")
#df_quality=df_merged['title_text'].str.lower().contains("quality")
#df_support=df_merged['title_text'].str.lower().contains("support")
#df_pricing=df_merged['title_text'].strlower().contains("pricing")
df.head()

,EcommerceSite,ReviewId,title_text,Rating,Review
0,Flipkart.com,925076148,Flipkart not provide service,3,Flipkart not delivered thinks in even smoll to...
1,Flipkart.com,925076148,Account has not working,3,Dear sirMy account not working please re-activ...
2,Flipkart.com,925076148,Painted black rice,1,POISON! When I opened the packet there was a s...
3,Flipkart.com,925076148,Its a Flopkart,1,Mostly you are on luck when you order from Fli...
4,Flipkart.com,925076148,Flipkart failed to deliver my order,1,Flipkart failed to deliver my order twice and ...


In [8]:
df.dropna(inplace = True)

In [9]:
#Label as 0-Negative, 1-Neutral, 2-Positive
df['Label'] = df["Rating"].apply(lambda x: 2 if x>3 else 1 if x==3 else 0)

In [10]:
df.drop('Rating', axis = 1, inplace = True)

In [11]:
df

,EcommerceSite,ReviewId,title_text,Review,Label
0,Flipkart.com,925076148,Flipkart not provide service,Flipkart not delivered thinks in even smoll to...,1
1,Flipkart.com,925076148,Account has not working,Dear sirMy account not working please re-activ...,1
2,Flipkart.com,925076148,Painted black rice,POISON! When I opened the packet there was a s...,0
3,Flipkart.com,925076148,Its a Flopkart,Mostly you are on luck when you order from Fli...,0
4,Flipkart.com,925076148,Flipkart failed to deliver my order,Flipkart failed to deliver my order twice and ...,0
...,...,...,...,...,...
1595,Flipkart.com,925076148,Most Distrustful !,I ordered a Spring mattress from Flipkart and ...,0
1596,Flipkart.com,925076148,Flipkart one of the great online shopping market,Flipkart very common and useful shopping marke...,1
1597,Flipkart.com,925076148,Wah Re Flipkart,New Sale-New Attraction-New Marketing-Surprisi...,0
1598,Flipkart.com,925076148,About flipkart,Flipkart is good shopping app now a days.peopl...,1


In [12]:
df_train = df[:19000]
df_test = df[19000:]

In [13]:
DATA_COLUMN = 'Review'
LABEL_COLUMN = 'Label'

label_list = [0, 1, 2]

In [14]:
train_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


In [15]:
train_InputExamples

0       <bert.run_classifier.InputExample object at 0x...
1       <bert.run_classifier.InputExample object at 0x...
2       <bert.run_classifier.InputExample object at 0x...
3       <bert.run_classifier.InputExample object at 0x...
4       <bert.run_classifier.InputExample object at 0x...
                              ...                        
1595    <bert.run_classifier.InputExample object at 0x...
1596    <bert.run_classifier.InputExample object at 0x...
1597    <bert.run_classifier.InputExample object at 0x...
1598    <bert.run_classifier.InputExample object at 0x...
1599    <bert.run_classifier.InputExample object at 0x...
Length: 1600, dtype: object

In [16]:
#This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return tokenization.FullTokenizer(
      vocab_file=vocab_file, 
      do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['flip', '##kar', '##t', 'not', 'delivered', 'thinks', 'in', 'even', 'sm', '##oll', 'town', 'i', 'orders', 'thinks', 'but', 'they', 'not', 'takes', 'my', 'order', '.', 'but', 'flip', '##kar', '##t', 'provided', 'good', 'service', 'in', 'city', "'", 's', 'they', 'gives', 'good', 'offers', 'on', 'it', '##ea', '##m', 'compare', 'other', 'online', 'website', '.', 'my', 'suggestion', 'if', 'u', 'live', 'city', "'", 's', 'best', 'buy', 'electronic', 'thinks', 'in', 'flip', '##kar', '##t', '.', 'they', 'will', 'gave', 'service', 'in', 'village', 'that', '##s', 'helps', 'to', 'poor', 'people', 'also', '##fl', '##ag', 'this', 'review', '##ir', '##rel', '##eva', '##nt', '##fa', '##ke', '##jun', '##k', 'thank', 'you', '!', 'we', 'appreciate', 'your', 'effort', '.']


In [18]:
#We'll set sequences to be at most 200 tokens long.
MAX_SEQ_LENGTH = 200

#Convert our train and validation features to InputFeatures that BERT can understand
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1600


INFO:tensorflow:Writing example 0 of 1600


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [19]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  Flipkart not delivered thinks in even smoll town I orders thinks but they not takes my order. But flipkart provided good service in city's they gives good offers on iteam compare other online website. my suggestion if u live city's best buy electronic thinks in flipkart. they will gave service in village thats helps to poor people alsoFlag This ReviewIrrelevantFakeJunk   Thank You! We appreciate your effort.
------------------------------
Tokens :  ['flip', '##kar', '##t', 'not', 'delivered', 'thinks', 'in', 'even', 'sm', '##oll', 'town', 'i', 'orders', 'thinks', 'but', 'they', 'not', 'takes', 'my', 'order', '.', 'but', 'flip', '##kar', '##t', 'provided', 'good', 'service', 'in', 'city', "'", 's', 'they', 'gives', 'good', 'offers', 'on', 'it', '##ea', '##m', 'compare', 'other', 'online', 'website', '.', 'my', 'suggestion', 'if', 'u', 'live', 'city', "'", 's', 'best', 'buy', 'electronic', 'thinks', 'in', 'flip', '##kar', '##t', '.', 'they', 'will', 'gave', 'service', 'in', '

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [21]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0

# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1

# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir='/content/bert-multiclass-sentiment',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)


In [23]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': '/content/bert-multiclass-sentiment', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19658e3890>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/bert-multiclass-sentiment', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19658e3890>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
#Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

#Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [25]:
#Training the model
print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


Training took time  0:00:00.016083


In [26]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving dict for global step 150: eval_accuracy = 0.884375, false_negatives = 62.0, false_positives = 10.0, global_step = 150, loss = 0.32362354, true_negatives = 1135.0, true_positives = 393.0


INFO:tensorflow:Saving dict for global step 150: eval_accuracy = 0.884375, false_negatives = 62.0, false_positives = 10.0, global_step = 150, loss = 0.32362354, true_negatives = 1135.0, true_positives = 393.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/bert-multiclass-sentiment/model.ckpt-150


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/bert-multiclass-sentiment/model.ckpt-150


{'eval_accuracy': 0.884375,
 'false_negatives': 62.0,
 'false_positives': 10.0,
 'global_step': 150,
 'loss': 0.32362354,
 'true_negatives': 1135.0,
 'true_positives': 393.0}

In [27]:
#To make predictions
def getPrediction(in_sentences):
  #A list to map the actual labels to the predictions
  labels = ["Negative", "Neutral","Positive"]

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [28]:
pred_sentences = list(df_test['Review'])

In [29]:
predictions = getPrediction(pred_sentences)

In [30]:
predictions

[]

In [31]:
enc_labels = []
act_labels = []
for i in range(len(predictions)):
  enc_labels.append(predictions[i][2])
  act_labels.append(predictions[i][3])

In [32]:
enc_labels[:5]

[]

In [33]:
df_test.Label[:5]

Series([], Name: Label, dtype: int64)

In [34]:
pd.DataFrame(enc_labels, columns = ['Label']).to_excel('bert-test-result.xlsx', index = False)

In [35]:
#Classifying random sentences
tests = getPrediction(["I've done several purchase from this app. But it's so disappointing at this moment. I've purchased a celeing fan and recived a wrong product so I complained and ask them to replace it. The next delivery I got the same wrong product with new packing. They are doing the same mistake againand  again. So I decided to give back this product and refund my amount. Still now I don't receive any calls or response from there part. It's like a cheeting. And this is the worst experience for me from this app.  I think Amazon I'd better than this. ..Flag This ReviewIrrelevantFakeJunk   Thank You! We appreciate your effort.",
                       "BriefFlipkart India's one of the biggest e-commerce company and it's service and Delivery is Also satisfying for its Consumers and mostly Deliverse on Time in Today's Review we Will take a of My Recent Experience From Flipkart.ReviewRecently I have ordered some study Material from Flipkart Like Table . The Offers they were providing is better then other Companies like Amazon and Other E-commerce WebsitesBut it take a Long period 8-12 days of time for Delivery which is not Acceptable but I don't have other choice I have to go for it cause my Budget range is Limited . After Receiving the product I feel so easy and Happy and the packaging is good but my experience of Flipkart is Not that Good so I recommend you to read the Conclusion part.ConclusionDelivery service is worst . You can shop from Flipkart if you have so much time or it only takes 2-3 for Delivery if not please don't shop from it use alternatives like Amazon and Etc.Stay safe Stay IndoorsPeace.Flag This ReviewIrrelevantFakeJunk   Thank You! We appreciate your effort.",
                       "Flipkart is one of the Top E-commerce website in India. Flipkart has always provided me fastet delivery within 3 days, (I belong to rural region, 3 days delivery is also fast for my location)The differs Flipkart from Other E-commerce website is their Promocodes, offers high discounts. Probably 95% of all my Online shopping is done through Flipkart.!Most of the times on other sites Price is shown more than MRP, But on Flipkart it is Same or Less than that MRP. Moreover all the needed products are available on Flipkart at affordable price.!Flipkart also offers you an opportunity for earning through affiliate marketing.Most of the times Payment is also a big issue. Flipkart offers a wide variety of payment methods and also refund is done within the stated time.Flipkart cares for the customer much seriously, takes feedback each and every time and solves the issues. This impressed me alot. O had given a suggestion to Flipkart, thought they will just read it and leave. But they read and implemented that suggestion. Amazing.!This is another kind of happiness that we can't express. A Billion dollar company takes your opinion and implements it within weeks.!Flipkart has Super Coin system which is also very good, my friend orders alot high prices products and gets super coins in respective ratios. Those products if had purchased from market will had cost him nearly 20-30% higher than purchased from Flipkart at discounted price. Moreover he had got alot super coins. Simply Sone pe Suhaga.!Flag This ReviewIrrelevantFakeJunk   Thank You! We appreciate your effort."
                       ])

INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert-multiclass-sentiment/model.ckpt-150


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [36]:
tests

[("I've done several purchase from this app. But it's so disappointing at this moment. I've purchased a celeing fan and recived a wrong product so I complained and ask them to replace it. The next delivery I got the same wrong product with new packing. They are doing the same mistake againand  again. So I decided to give back this product and refund my amount. Still now I don't receive any calls or response from there part. It's like a cheeting. And this is the worst experience for me from this app.  I think Amazon I'd better than this. ..Flag This ReviewIrrelevantFakeJunk   Thank You! We appreciate your effort.",
  array([-0.03853464, -3.4936862 , -4.9045134 ], dtype=float32),
  0,
  'Negative'),
 ("BriefFlipkart India's one of the biggest e-commerce company and it's service and Delivery is Also satisfying for its Consumers and mostly Deliverse on Time in Today's Review we Will take a of My Recent Experience From Flipkart.ReviewRecently I have ordered some study Material from Flipkart